In [1]:
class Rotor:
    def __init__(self, wiring, notch, position=0):
        """
        Constructor para el rotor.
        :param wiring: Cadena que representa el mapeo interno del rotor.
        :param notch: Posición de la muesca del rotor.
        :param position: Posición inicial del rotor.
        """
        self.wiring = wiring  # Almacena el mapeo interno del rotor
        self.notch = notch  # Almacena la posición de la muesca del rotor
        self.position = position  # Almacena la posición inicial del rotor

    def rotate(self):
        """
        Rota el rotor y verifica si alcanza la muesca.
        :return: True si el rotor alcanza la muesca, de lo contrario False.
        """
        self.position = (self.position + 1) % 26  # Incrementa la posición del rotor y la ajusta al rango [0-25]
        return self.position == self.notch  # Devuelve True si el rotor alcanza la muesca

    def encrypt_forward(self, char):
        """
        Cifra una letra al pasar por el rotor en la dirección hacia adelante.
        :param char: Letra a cifrar.
        :return: Letra cifrada.
        """
        index = (ord(char) - ord('A') + self.position) % 26  # Calcula el índice ajustado de la letra
        encrypted_char = self.wiring[index]  # Obtiene la letra cifrada del mapeo interno
        return chr((ord(encrypted_char) - ord('A') - self.position + 26) % 26 + ord('A'))  # Ajusta y devuelve la letra cifrada

    def encrypt_backward(self, char):
        """
        Cifra una letra al pasar por el rotor en la dirección hacia atrás.
        :param char: Letra a cifrar.
        :return: Letra cifrada.
        """
        index = (self.wiring.index(char) - self.position + 26) % 26  # Encuentra el índice de la letra y ajusta la posición
        return chr((index + self.position) % 26 + ord('A'))  # Ajusta y devuelve la letra cifrada

class Reflector:
    def __init__(self, wiring):
        """
        Constructor para el reflector.
        :param wiring: Cadena que representa el mapeo del reflector.
        """
        self.wiring = wiring  # Almacena el mapeo del reflector

    def reflect(self, char):
        """
        Refleja una letra.
        :param char: Letra a reflejar.
        :return: Letra reflejada.
        """
        return self.wiring[ord(char) - ord('A')]  # Devuelve la letra reflejada del mapeo

def encrypt_message(message, rotors, reflector, steckerbrett):
    """
    Cifra un mensaje utilizando los rotores, el reflector y el panel de conexiones.
    :param message: Mensaje a cifrar.
    :param rotors: Lista de rotores.
    :param reflector: Reflector.
    :param steckerbrett: Diccionario del panel de conexiones.
    :return: Mensaje cifrado.
    """
    encrypted_message = ""  # Inicializa el mensaje cifrado
    for char in message:
        if char.isalpha():  # Solo procesa caracteres alfabéticos
            char = char.upper()  # Convierte la letra a mayúscula
            if char in steckerbrett:
                char = steckerbrett[char]  # Aplica el panel de conexiones antes de los rotores
            for rotor in rotors:
                char = rotor.encrypt_forward(char)  # Pasa la letra por los rotores hacia adelante
            char = reflector.reflect(char)  # Refleja la letra
            for rotor in reversed(rotors):
                char = rotor.encrypt_backward(char)  # Pasa la letra por los rotores hacia atrás
            if char in steckerbrett:
                char = steckerbrett[char]  # Aplica el panel de conexiones después de los rotores
            if rotors[0].rotate():  # Rota el primer rotor y propaga la rotación si es necesario
                for i in range(1, len(rotors)):
                    if not rotors[i-1].rotate():
                        break
        encrypted_message += char  # Agrega la letra cifrada al mensaje cifrado
    return encrypted_message  # Devuelve el mensaje cifrado

def configurar_y_cifrar_mensaje():
    """
    Configura los rotores, el reflector y el panel de conexiones, y cifra un mensaje ingresado por el usuario.
    """
    # Solicitar al usuario la cantidad de rotores
    while True:
        num_rotors = int(input("Ingrese el número de rotores (3-5): "))  # Solicita el número de rotores
        if 3 <= num_rotors <= 5:
            break
        else:
            print("Número de rotores debe estar entre 3 y 5.")

    # Configuración de los rotores
    rotor_wirings = [
        "EKMFLGDQVZNTOWYHXUSPAIBRCJ",  # Mapeo del rotor 1
        "AJDKSIRUXBLHWTMCQGZNPYFVOE",  # Mapeo del rotor 2
        "BDFHJLCPRTXVZNYEIWGAKMUSQO",  # Mapeo del rotor 3
        "ESOVPZJAYQUIRHXLNFTGKDCMWB",  # Mapeo del rotor 4
        "VZBRGITYUPSDNHLXAWMJQOFECK"   # Mapeo del rotor 5
    ]
    rotor_notches = [16, 4, 21, 9, 25]  # Muescas de los rotores
    
    rotors = []  # Lista para almacenar los rotores configurados
    for i in range(num_rotors):
        wiring = rotor_wirings[i]  # Obtiene el mapeo del rotor
        notch = rotor_notches[i]  # Obtiene la muesca del rotor
        position = int(input(f"Ingrese la posición inicial del rotor {i + 1} (0-25): "))  # Solicita la posición inicial del rotor
        rotors.append(Rotor(wiring, notch, position))  # Crea y agrega el rotor a la lista

    # Configuración del reflector
    reflector = Reflector("YRUHQSLDPXNGOKMIEBFZCWVJAT")  # Crea el reflector con el mapeo especificado

    # Configuración del panel de conexiones (Steckerbrett)
    steckerbrett = {}  # Diccionario para almacenar las conexiones
    num_conexiones = int(input("Ingrese el número de conexiones del panel (0-10): "))  # Solicita el número de conexiones
    
    for _ in range(num_conexiones):
        while True:
            conexion = input("Ingrese la conexión (formato A:B): ")  # Solicita la conexión en el formato especificado
            entrada, salida = conexion.split(":")  # Divide la conexión en entrada y salida
            if entrada in steckerbrett or salida in steckerbrett:
                print(f"Ya existe una conexión para {entrada} o {salida}.")
                reemplazar = input("¿Desea reemplazarla? (s/n): ").lower()
                if reemplazar == 's':
                    if entrada in steckerbrett:
                        steckerbrett.pop(steckerbrett[entrada], None)  # Elimina la conexión existente
                        steckerbrett.pop(entrada, None)  # Elimina la entrada del diccionario
                    if salida in steckerbrett:
                        steckerbrett.pop(steckerbrett[salida], None)  # Elimina la conexión existente
                        steckerbrett.pop(salida, None)  # Elimina la salida del diccionario
                else:
                    continue
            steckerbrett[entrada] = salida  # Agrega la nueva conexión al diccionario
            steckerbrett[salida] = entrada  # Agrega la conexión inversa al diccionario
            break

    # Solicitar al usuario el mensaje a cifrar
    message = input("Ingrese el mensaje a cifrar: ")  # Solicita el mensaje a cifrar
    encrypted_message = encrypt_message(message, rotors, reflector, steckerbrett)  # Cifra el mensaje
    print("Mensaje cifrado:", encrypted_message)  # Imprime el mensaje cifrado

# Llamar a la función para configurar y cifrar el mensaje
configurar_y_cifrar_mensaje()  # Ejecuta la función principal para configurar y cifrar el mensaje

Ya existe una conexión para A o C.
Mensaje cifrado: NOZOZR
